In [44]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import cufflinks as cf
from math import log, sqrt
import datetime as dt
from sklearn.linear_model import LinearRegression

In [45]:
dfq = pd.read_excel('../EMCreditRV/CDS.xlsx', index_col=0, parse_dates=True)
countries = []
for c in dfq.columns:
    countries.append(c.split()[0])
dfq.columns = countries

aggregated_df = dfq[['BRAZIL', 'COLOM', 'REPSOU']].copy()

In [46]:
dfq = pd.read_excel('../EMRatesPCA/SA.xlsx', index_col=0, parse_dates=True)
aggregated_df[['ZAR 2Y', 'ZAR 10Y']] = dfq[['2Y', '10Y']]

In [47]:
dfq = pd.read_excel('../../FXVolStatArb/1Yfxvol.xlsx', index_col=0, parse_dates=True)
aggregated_df[['EURPLNV', 'EURHUFV', 'USDMXNV', 'USDCADV']] = dfq[['EURPLN', 'EURHUF', 'USDMXN', 'USDCAD']]

In [121]:
lookback = 100
sample_df = aggregated_df.diff().dropna()[['BRAZIL', 'COLOM', 'ZAR 2Y', 'ZAR 10Y',
                                        'EURPLNV', 'EURHUFV', 'USDMXNV', 'USDCADV']].head(lookback)
sample = sample_df.values

dfmacro = pd.read_excel('macro.xlsx', index_col=0, parse_dates=True)
dfret = dfmacro[['SPX Index', 'BCOM Index', 'DXY Curncy']].pct_change().dropna().loc[sample_df.index]
dfabs = dfmacro[['VIX Index', 'USGG10YR Index', 'USOHHYTO Index']].diff().dropna().loc[sample_df.index]

In [113]:
#FX trades VaR
dffx = pd.read_excel('FX.xlsx', index_col=0, parse_dates=True)
dffx.columns = ['USDPLN', 'USDBRL']
fx_sample = dffx.pct_change().dropna().head(lookback).values
fx_weights = [10000000, -5000000]
print('99% Daily VaR:', np.percentile(fx_sample.dot(fx_weights), 1))
print('95% Daily VaR:', np.percentile(fx_sample.dot(fx_weights), 5))
print('$ Volatility:', sqrt(260) * fx_sample.dot(fx_weights).std())

99% Daily VaR: -194405.7885735425
95% Daily VaR: -110716.0420646291
$ Volatility: 1154422.2569902334


In [120]:
#Credit trades VaR
sample = aggregated_df.diff().dropna()[['BRAZIL', 'COLOM']].head(lookback).values
size = 10000
weights = [1.6 * size, -1.0 * size]
print('99% Daily VaR:', np.percentile(sample.dot(weights), 1))
print('95% Daily VaR:', np.percentile(sample.dot(weights), 5))
print('$ Volatility:', sqrt(260) * sample.dot(weights).std())
size = 20/60
weights = [1.6 * size, -1.0 * size]
print('Daily VaR cost of $1:', np.percentile(sample.dot(weights), 1))
size = 1/60
weights = [1.6 * size, -1.0 * size]
print('Total VaR cost of 1$:', sqrt(20) * np.percentile(sample.dot(weights), 1))

99% Daily VaR: -221428.4399999999
95% Daily VaR: -81503.09999999989
$ Volatility: 1356755.9993392592
Daily VaR cost of $1: -7.3809479999999965
Total VaR cost of 1$: -1.6504301466391116


In [115]:
x = np.concatenate([dfret.values, dfabs.values], axis=1)
y = sample.dot(weights)
reg = LinearRegression().fit(x, y)
for i in range(len(reg.coef_)):
    print(dfret.columns.append(dfabs.columns)[i], reg.coef_[i])

SPX Index -2.8789018748209165
BCOM Index -1.7843657822006762
DXY Curncy -0.14589818359631143
VIX Index 0.009998039849547946
USGG10YR Index -0.1156899949417203
USOHHYTO Index 0.000197791552764226


In [116]:
#Rates trades VaR
sample = 100 * aggregated_df.diff().dropna()[['ZAR 2Y', 'ZAR 10Y']].head(lookback).values
size = 10000
weights = [1 * size, -1.8 * size]
print('99% Daily VaR:', np.percentile(sample.dot(weights), 1))
print('95% Daily VaR:', np.percentile(sample.dot(weights), 5))
print('$ Volatility:', sqrt(260) * sample.dot(weights).std())
size = 60/80
weights = [1 * size, -1.8 * size]
print('VaR cost of $1:', np.percentile(sample.dot(weights), 1))
size = 1/90
weights = [1 * size, -1.8 * size]
print('Total VaR cost of 1$:', sqrt(60) * np.percentile(sample.dot(weights), 1))

99% Daily VaR: -265024.9999999993
95% Daily VaR: -164149.99999999974
$ Volatility: 1822372.0730013906
VaR cost of $1: -19.87687499999995
Total VaR cost of 1$: -2.2809720251747074


In [10]:
x = np.concatenate([dfret.values, dfabs.values], axis=1)
y = sample.dot(weights)
reg = LinearRegression().fit(x, y)
for i in range(len(reg.coef_)):
    print(dfret.columns.append(dfabs.columns)[i], reg.coef_[i])

SPX Index -2.891054286940919
BCOM Index 1.8484908372468039
DXY Curncy -3.0368382580711564
VIX Index -0.005916848867129121
USGG10YR Index -0.6679075765724866
USOHHYTO Index -0.00591235341332963


In [117]:
#FXVol trades VaR
sample = aggregated_df.diff().dropna()[['EURPLNV', 'EURHUFV', 'USDMXNV', 'USDCADV']].head(lookback).values
size = 250000
weights = [0.5 * size, 0.25 * size, 0.25 * size, -2 * size]
print('99% Daily VaR:', np.percentile(sample.dot(weights), 1))
print('95% Daily VaR:', np.percentile(sample.dot(weights), 5))
print('$ Volatility:', sqrt(260) * sample.dot(weights).std())
size = 20 / 1.2
weights = [0.5 * size, 0.25 * size, 0.25 * size, -2 * size]
print('VaR cost of $1:', np.percentile(sample.dot(weights), 1))
size = 1/1.2
weights = [0.5 * size, 0.25 * size, 0.25 * size, -2 * size]
print('Total VaR cost of 1$:', sqrt(20) * np.percentile(sample.dot(weights), 1))

99% Daily VaR: -173295.31249999994
95% Daily VaR: -79374.9999999998
$ Volatility: 1029680.4293135166
VaR cost of $1: -11.553020833333328
Total VaR cost of 1$: -2.5833339928804597


In [12]:
x = np.concatenate([dfret.values, dfabs.values], axis=1)
y = sample.dot(weights)
reg = LinearRegression().fit(x, y)
for i in range(len(reg.coef_)):
    print(dfret.columns.append(dfabs.columns)[i], reg.coef_[i])

SPX Index 3.4239075077649495
BCOM Index -0.2523751555363134
DXY Curncy -3.625161989563399
VIX Index 0.0031957614230838163
USGG10YR Index 0.14377530453768345
USOHHYTO Index -0.0003904504638361783


In [122]:
#Total VaR
sample = sample_df.values
size1 = 10000   #Credit
size2 = 375000  #Rates
size3 = 250000  #FXVol
weights = [1.6 * size1, -1.0 * size1, 1 * size2, -1.8 * size2, 0.5 * size3, 0.25 * size3, 0.25 * size3, -2 * size3]
fx_sample = dffx.pct_change().dropna().loc[sample_df.index].values
fx_weights = [10000000, -5000000]
print('99% Daily VaR:', np.percentile(sample.dot(weights) + fx_sample.dot(fx_weights), 1))
print('95% Daily VaR:', np.percentile(sample.dot(weights) + fx_sample.dot(fx_weights), 5))
print('$ Volatility:', sqrt(260) * (sample.dot(weights)+ fx_sample.dot(fx_weights)).std())

99% Daily VaR: -167638.23197003859
95% Daily VaR: -119863.02529872966
$ Volatility: 1231908.125793943


In [123]:
dfmacro = pd.read_excel('macro.xlsx', index_col=0, parse_dates=True)
dfret = dfmacro[['SPX Index', 'BCOM Index', 'DXY Curncy']].pct_change().dropna().loc[sample_df.index]
dfabs = dfmacro[['VIX Index', 'USGG10YR Index', 'USOHHYTO Index']].diff().dropna().loc[sample_df.index]

In [93]:
x = np.concatenate([dfret.values, dfabs.values], axis=1)
y = fx_sample.dot(fx_weights)
reg = LinearRegression().fit(x, y)
for i in range(len(reg.coef_)):
    print(dfret.columns.append(dfabs.columns)[i], reg.coef_[i])

SPX Index -300239.1117089246
BCOM Index 1837729.9254541248
DXY Curncy 12840397.65862339
VIX Index 369.36772318260046
USGG10YR Index -214461.3602051424
USOHHYTO Index -805.7295304762665


In [94]:
x = np.concatenate([dfret.values, dfabs.values], axis=1)
y = sample.dot(weights)
reg = LinearRegression().fit(x, y)
for i in range(len(reg.coef_)):
    print(dfret.columns.append(dfabs.columns)[i], reg.coef_[i])

SPX Index -456849.9079998917
BCOM Index 490461.94029133173
DXY Curncy -1742356.0569253531
VIX Index 717.8850890193135
USGG10YR Index -155787.91018357192
USOHHYTO Index -1784.307385284861


In [124]:
x = np.concatenate([dfret.values, dfabs.values], axis=1)
y = sample.dot(weights) + fx_sample.dot(fx_weights)
reg = LinearRegression().fit(x, y)
for i in range(len(reg.coef_)):
    print(dfret.columns.append(dfabs.columns)[i], reg.coef_[i])

SPX Index -884161.8543305196
BCOM Index 2313769.0730908113
DXY Curncy 11190572.964708712
VIX Index 1526.2761008917587
USGG10YR Index -363624.6940016283
USOHHYTO Index -2507.975158045534


In [21]:
3000 / 0.8

3750.0

In [22]:
3750 * 1.8

6750.0

In [59]:
3000 / 0.4

7500.0

In [76]:
dfq = pd.read_excel('../EMRatesPCA/SA.xlsx', index_col=0, parse_dates=True)
aggregated_df[['ZAR 3Y', 'ZAR 7Y']] = dfq[['3Y', '7Y']]
sample = 100 * aggregated_df.diff().dropna()[['ZAR 3Y', 'ZAR 7Y']].head(lookback).values
size = 7500
weights = [1 * size, -1.4 * size]
print('99% Daily VaR:', np.percentile(sample.dot(weights), 1))
print('95% Daily VaR:', np.percentile(sample.dot(weights), 5))
print('$ Volatility:', sqrt(260) * sample.dot(weights).std())

99% Daily VaR: -81963.75000000095
95% Daily VaR: -69431.25000000159
$ Volatility: 687818.3251229966


In [86]:
#Total VaR
sample = aggregated_df.diff().dropna()[['BRAZIL', 'COLOM', 'ZAR 3Y', 'ZAR 7Y',
                                        'EURPLNV', 'EURHUFV', 'USDMXNV', 'USDCADV']].head(lookback).values
size1 = 8000
size2 = 750000
size3 = 250000
weights = [1.6 * size1, -1.0 * size1, 1 * size2, -1.4 * size2, 0.5 * size3, 0.25 * size3, 0.25 * size3, -2 * size3]
fx_sample = dffx.pct_change().dropna().loc[sample_df.index].values
fx_weights = [10000000, -5000000]
print('99% Daily VaR:', np.percentile(sample.dot(weights) + fx_sample.dot(fx_weights), 1))
print('95% Daily VaR:', np.percentile(sample.dot(weights) + fx_sample.dot(fx_weights), 5))
print('$ Volatility:', sqrt(260) * (sample.dot(weights)+ fx_sample.dot(fx_weights)).std())

99% Daily VaR: -194271.6929704755
95% Daily VaR: -120153.47546758693
$ Volatility: 1265689.6322579829


In [87]:
x = np.concatenate([dfret.values, dfabs.values], axis=1)
y = sample.dot(weights) + fx_sample.dot(fx_weights)
reg = LinearRegression().fit(x, y)
for i in range(len(reg.coef_)):
    print(dfret.columns.append(dfabs.columns)[i], reg.coef_[i])

SPX Index -333138.01582833455
BCOM Index 2411042.3758058636
DXY Curncy 10248631.749656849
VIX Index -347.59162787446985
USGG10YR Index -303917.4099047304
USOHHYTO Index -1467.8962120073847


In [89]:
3750 * 90, 40 * 7500

(337500, 300000)

In [125]:
3750 * 1.8

6750.0

In [126]:
6750 * 600000000 / 18000

225000000.0

In [127]:
20000 / 8

2500.0

In [130]:
16 * 30000000 / 12.8

37500000.0

In [131]:
3750 * 35

131250